In [3]:
import pandas as pd
import quandl

# Install the quandl package if it's not already installed
# !pip install quandl

# Set the API key
quandl.ApiConfig.api_key = "xyrEL2EBKf8XvGzp1YdA"

# Load the CSV file
robinhood_profit_df = pd.read_csv('Robinhood_Profit.csv')

# Extract the unique tickers from the CSV file
tickers = robinhood_profit_df['Instrument'].unique()

# Function to check if a ticker is valid
def is_valid_ticker(ticker):
    try:
        # Try fetching a single row of data to validate the ticker
        quandl.get(f"WIKI/{ticker}", rows=1)
        return True
    except Exception:
        return False

# Filter out invalid tickers
valid_tickers = [ticker for ticker in tickers if is_valid_ticker(ticker)]

# Prepare an empty DataFrame to store the data
data_frames = []

# Fetch the data for each valid ticker and store it in the DataFrame
for ticker in valid_tickers:
    try:
        # Fetch the current data for the ticker
        stock_data = quandl.get(f"WIKI/{ticker}", rows=1)
        stock_data['Ticker'] = ticker
        data_frames.append(stock_data)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Concatenate all the individual DataFrames into one
if data_frames:
    data = pd.concat(data_frames, ignore_index=True)
    # Display the collected data
    print(data)
else:
    print("No valid data fetched.")

# Check if the data was collected and concatenate it into a DataFrame
if data_frames:
    data = pd.concat(data_frames, ignore_index=True)
    # Display the collected data
    print(data)
    data.to_csv('Collected_Stock_Data.csv', index=False)
else:
    print("No valid data fetched.")


       Open      High      Low     Close     Volume  Ex-Dividend  Split Ratio  \
0    1.8000    1.8900    1.700    1.8492    72531.0          0.0          1.0   
1   10.6900   10.8600   10.451   10.5300   557983.0          0.0          1.0   
2  140.5000  140.5700  140.500  140.5200  1189653.0          0.0          1.0   
3    3.4672    3.4672    3.350    3.3900     1925.0          0.0          1.0   
4    3.0900    3.2000    2.970    3.1100  1222100.0          0.0          1.0   
5    2.3600    2.4390    2.220    2.2600  2135214.0          0.0          1.0   
6    2.8000    2.8300    2.670    2.7100   120485.0          0.0          1.0   

   Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume Ticker  
0     1.8000     1.8900     1.700      1.8492      72531.0   BPTH  
1    10.6900    10.8600    10.451     10.5300     557983.0   SNCR  
2   140.5000   140.5700   140.500    140.5200    1189653.0    PRE  
3     3.4672     3.4672     3.350      3.3900       1925.0   MIND  
4     3.090

<h3>------------------ARDA ADJUSTMENT------------------<h>

In [19]:
import pandas as pd
import quandl

# Set the API key
quandl.ApiConfig.api_key = "xyrEL2EBKf8XvGzp1YdA"

# Function to check if a ticker has a split ratio other than 1
def get_data_with_split_ratio(ticker):
    try:
        # Fetch data for the ticker
        stock_data = quandl.get(f"WIKI/{ticker}")
        # Filter to keep only the columns of interest
        stock_data = stock_data[['Adj. Close', 'Split Ratio']]
        stock_data['Ticker'] = ticker
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
    return None

# Prepare an empty DataFrame to store the data
data_frames = []

# Function to fetch tickers in chunks and process them
def fetch_and_process_tickers(chunk_size=10, max_rows=10):
    try:
        start = 0
        while len(data_frames) < max_rows:
            # Fetch the list of tickers in chunks
            metadata = quandl.get_table('WIKI/PRICES', qopts={'columns': ['ticker']}, paginate=True)
            tickers = metadata['ticker'].unique()[start:start + chunk_size]
            start += chunk_size
            
            # Fetch the data for each ticker and store it in the DataFrame
            for ticker in tickers:
                data = get_data_with_split_ratio(ticker)
                if data is not None:
                    data_frames.append(data)
                # Stop if we have reached the desired number of rows
                if len(data_frames) >= max_rows:
                    break
            # Break outer loop if we have reached the desired number of rows
            if len(data_frames) >= max_rows:
                break
    except Exception as e:
        print(f"Error fetching metadata: {e}")

# Fetch and process tickers
fetch_and_process_tickers()

# Concatenate all the individual DataFrames into one
if data_frames:
    result_data = pd.concat(data_frames, ignore_index=True)
    # Limit the result to 1000 rows
    result_data = result_data.head(1000)
    # Display the collected data
    print(result_data)
    # Export the collected data to a CSV file
    result_data.to_csv('Stocks_With_Split_Ratios.csv', index=False)
else:
    print("No stocks found.")


Error fetching metadata: This call exceeds the amount of data that quandl.get_table() allows.         Please use the following link in your browser, which will download the full results as         a CSV file: https://www.quandl.com/api/v3/datatables/WIKI/PRICES?qopts.export=true&api_key=xyrEL2EBKf8XvGzp1YdA . See         our API documentation for more info:         https://docs.quandl.com/docs/in-depth-usage-1#section-download-an-entire-table
No stocks found.


In [10]:
import requests

# Set the API key
api_key = "xyrEL2EBKf8XvGzp1YdA"

# Define the request URL for the WIKI dataset to get column names
url = f"https://data.nasdaq.com/api/v3/datatables/WIKI/PRICES.json?qopts.columns=ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume&api_key={api_key}"

# Make the request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Extract and print the columns
    columns = data['datatable']['columns']
    column_names = [column['name'] for column in columns]
    print("Available columns:", column_names)
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")


Available columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'ex-dividend', 'split_ratio', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume']


In [29]:
from bs4 import BeautifulSoup
import requests

# URL of the webpage that contains stock split data
url = 'https://example.com/stock-splits'  # Replace with the actual URL

# Custom headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Making the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Adjust the class names and tags based on the actual HTML structure
    splits_data = soup.find_all('div', class_='split-data-class')
    for split in splits_data:
        print(split.text)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Failed to retrieve data. Status code: 500
